# pandas基礎：データ操作・分析入門

このノートブックでは、**pandas**を使った基本的なデータ操作と分析手法を学習します。

## 学習内容
1. pandasの基本設定とインポート
2. サンプルデータの作成とDataFrame生成
3. CSVファイルからのデータ読み込み
4. データの基本情報確認
5. 列・行の抽出とフィルタリング
6. データの集計（groupby）
7. データの並び替えと欠損値処理
8. 簡単なグラフ描画

## 前提知識
- Python基礎文法
- リスト、辞書などの基本データ構造

## 1. pandasのインポートと基本設定

必要なライブラリをインポートし、pandasの表示設定を調整します。

In [ ]:
# 必要なライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings

# 表示設定
pd.set_option('display.max_columns', 10)
pd.set_option('display.max_rows', 20)
pd.set_option('display.width', 1000)

# Matplotlib日本語フォント設定（警告を抑制）
plt.rcParams['font.family'] = 'DejaVu Sans'
warnings.filterwarnings('ignore')

# Seabornスタイル設定
sns.set_style("whitegrid")

print("必要なライブラリがインポートされました！")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")

## 2. サンプルデータの作成とDataFrame生成

pandasのDataFrameを作成する方法を学習します。辞書やリストから直接DataFrameを作成してみましょう。

In [ ]:
# 辞書からDataFrameを作成
students_data = {
    '名前': ['田中太郎', '佐藤花子', '鈴木一郎', '高橋美咲', '山田次郎'],
    '年齢': [20, 19, 21, 20, 22],
    '学部': ['工学部', '文学部', '理学部', '経済学部', '法学部'],
    '数学': [85, 92, 78, 90, 88],
    '英語': [78, 88, 85, 92, 75],
    '国語': [90, 85, 88, 87, 92]
}

# DataFrameの作成
df_students = pd.DataFrame(students_data)

print("学生データのDataFrame:")
print(df_students)
print(f"\nDataFrameの形状: {df_students.shape}")  # (行数, 列数)
print(f"データ型:\n{df_students.dtypes}")

In [ ]:
# より複雑なサンプルデータの作成（売上データ）
np.random.seed(42)  # 再現性のため

# 日付データの生成
start_date = datetime(2024, 1, 1)
dates = [start_date + timedelta(days=i) for i in range(30)]

# ランダムデータの生成
sales_data = {
    '日付': np.random.choice(dates, 100),
    '商品名': np.random.choice(['商品A', '商品B', '商品C', '商品D', '商品E'], 100),
    'カテゴリ': np.random.choice(['電子機器', '衣料品', '食品', '書籍'], 100),
    '単価': np.random.randint(100, 1000, 100),
    '数量': np.random.randint(1, 10, 100),
    '地域': np.random.choice(['東京', '大阪', '名古屋', '福岡'], 100)
}

# DataFrameの作成
df_sales = pd.DataFrame(sales_data)

# 売上金額の計算（新しい列を追加）
df_sales['売上金額'] = df_sales['単価'] * df_sales['数量']

print("売上データのDataFrame（最初の10行）:")
print(df_sales.head(10))
print(f"\nDataFrameの形状: {df_sales.shape}")

# CSVファイルに保存（後で使用）
df_sales.to_csv('sample_sales_data.csv', index=False, encoding='utf-8')
print("\nデータを 'sample_sales_data.csv' に保存しました")

## 3. データの読み込み（CSVファイル）

pandasの`read_csv()`関数を使って、CSVファイルからデータを読み込む方法を学習します。

In [ ]:
# CSVファイルからデータを読み込み
try:
    df_loaded = pd.read_csv('sample_sales_data.csv', encoding='utf-8')
    print("CSVファイルからデータを読み込みました")
    print(f"読み込んだデータの形状: {df_loaded.shape}")
    print("\n最初の5行:")
    print(df_loaded.head())
    
except FileNotFoundError:
    print("CSVファイルが見つかりません。先にサンプルデータを作成してください。")
    df_loaded = df_sales  # 代替として先ほど作成したデータを使用

# 日付列をdatetime型に変換
df_loaded['日付'] = pd.to_datetime(df_loaded['日付'])

print(f"\n日付列のデータ型: {df_loaded['日付'].dtype}")
print(f"データ型一覧:\n{df_loaded.dtypes}")

## 4. データの基本情報確認

`shape`, `head()`, `info()`, `describe()`などを使って、データの基本的な情報を確認します。

In [ ]:
# データの基本情報確認
print("=== データの基本情報 ===")

# 1. データの形状
print(f"1. データの形状: {df_loaded.shape}")
print(f"   行数: {df_loaded.shape[0]}, 列数: {df_loaded.shape[1]}")

# 2. 列名の確認
print(f"\n2. 列名: {list(df_loaded.columns)}")

# 3. 最初の数行
print("\n3. 最初の5行:")
print(df_loaded.head())

# 4. 最後の数行
print("\n4. 最後の3行:")
print(df_loaded.tail(3))

# 5. データ型と欠損値の情報
print("\n5. データ型と欠損値の情報:")
print(df_loaded.info())

# 6. 統計サマリー
print("\n6. 数値列の統計サマリー:")
print(df_loaded.describe())

# 7. カテゴリ列のユニーク値
print("\n7. カテゴリ列のユニーク値:")
categorical_columns = ['商品名', 'カテゴリ', '地域']
for col in categorical_columns:
    if col in df_loaded.columns:
        print(f"   {col}: {df_loaded[col].unique()}")
        print(f"   ユニーク数: {df_loaded[col].nunique()}")

## 5. 列・行の抽出とフィルタリング

pandasでデータを抽出・フィルタリングする様々な方法を学習します。

In [ ]:
print("=== 列の抽出 ===")

# 1. 単一列の抽出
print("1. 単一列の抽出（商品名）:")
product_names = df_loaded['商品名']
print(f"データ型: {type(product_names)}")  # Series
print(product_names.head())

# 2. 複数列の抽出
print("\n2. 複数列の抽出（商品名、カテゴリ、売上金額）:")
selected_columns = df_loaded[['商品名', 'カテゴリ', '売上金額']]
print(f"データ型: {type(selected_columns)}")  # DataFrame
print(selected_columns.head())

# 3. 条件による列の抽出（数値列のみ）
print("\n3. 数値列のみを抽出:")
numeric_columns = df_loaded.select_dtypes(include=[np.number])
print(f"数値列: {list(numeric_columns.columns)}")
print(numeric_columns.head())

In [ ]:
print("\n=== 行の抽出・フィルタリング ===")

# 1. インデックスによる行の抽出
print("1. インデックスによる行の抽出:")
print("最初の3行 (iloc[0:3]):")
print(df_loaded.iloc[0:3])

print("\n特定の行 (iloc[5]):")
print(df_loaded.iloc[5])

# 2. 条件によるフィルタリング
print("\n2. 条件によるフィルタリング:")

# 売上金額が5000以上のデータ
high_sales = df_loaded[df_loaded['売上金額'] >= 5000]
print(f"売上金額5000以上のデータ: {len(high_sales)}件")
print(high_sales.head())

# 特定のカテゴリのデータ
electronics = df_loaded[df_loaded['カテゴリ'] == '電子機器']
print(f"\n電子機器カテゴリのデータ: {len(electronics)}件")
print(electronics.head(3))

# 複数条件（AND）
high_electronics = df_loaded[
    (df_loaded['カテゴリ'] == '電子機器') & 
    (df_loaded['売上金額'] >= 3000)
]
print(f"\n電子機器かつ売上3000以上: {len(high_electronics)}件")

# 複数条件（OR）
high_value = df_loaded[
    (df_loaded['売上金額'] >= 8000) | 
    (df_loaded['数量'] >= 8)
]
print(f"\n売上8000以上または数量8以上: {len(high_value)}件")

# isinを使用した条件
target_products = ['商品A', '商品B']
selected_products = df_loaded[df_loaded['商品名'].isin(target_products)]
print(f"\n商品AまたはBのデータ: {len(selected_products)}件")

## 6. データの集計（groupby、集約関数）

`groupby()`と集約関数を使って、データを分類・集計する方法を学習します。

In [ ]:
print("=== データの集計 ===")

# 1. カテゴリ別の基本集計
print("1. カテゴリ別の売上集計:")
category_sales = df_loaded.groupby('カテゴリ')['売上金額'].agg([
    'sum',    # 合計
    'mean',   # 平均
    'count',  # 件数
    'max',    # 最大値
    'min'     # 最小値
]).round(2)

print(category_sales)

# 2. 商品別の集計
print("\n2. 商品別の売上集計:")
product_sales = df_loaded.groupby('商品名')['売上金額'].sum().sort_values(ascending=False)
print(product_sales)

# 3. 地域別の集計
print("\n3. 地域別の集計:")
region_stats = df_loaded.groupby('地域').agg({
    '売上金額': ['sum', 'mean'],
    '数量': ['sum', 'mean'],
    '商品名': 'count'  # 取引件数
}).round(2)

print(region_stats)

# 4. 複数列でのグループ化
print("\n4. カテゴリ×地域別の売上:")
category_region = df_loaded.groupby(['カテゴリ', '地域'])['売上金額'].sum().unstack(fill_value=0)
print(category_region)

# 5. 集計結果の可視化用データ
print("\n5. 集計結果の保存:")
category_summary = df_loaded.groupby('カテゴリ').agg({
    '売上金額': 'sum',
    '数量': 'sum',
    '商品名': 'count'
}).round(2)

category_summary.columns = ['売上合計', '数量合計', '取引件数']
print(category_summary)

## 7. データの並び替えと欠損値処理

`sort_values()`による並び替えと、欠損値の確認・処理方法を学習します。

In [ ]:
print("=== データの並び替え ===")

# 1. 単一列による並び替え
print("1. 売上金額で降順に並び替え:")
sorted_by_sales = df_loaded.sort_values('売上金額', ascending=False)
print(sorted_by_sales[['商品名', 'カテゴリ', '売上金額']].head())

# 2. 複数列による並び替え
print("\n2. カテゴリ→売上金額で並び替え:")
sorted_multi = df_loaded.sort_values(['カテゴリ', '売上金額'], ascending=[True, False])
print(sorted_multi[['商品名', 'カテゴリ', '売上金額']].head(10))

# 3. インデックスのリセット
print("\n3. インデックスをリセット:")
sorted_reset = df_loaded.sort_values('売上金額', ascending=False).reset_index(drop=True)
print(sorted_reset[['商品名', 'カテゴリ', '売上金額']].head())

print("\n=== 欠損値の確認と処理 ===")

# まず、意図的に欠損値を作成してデモンストレーション
df_with_missing = df_loaded.copy()

# ランダムに欠損値を作成
np.random.seed(42)
missing_indices = np.random.choice(df_with_missing.index, size=10, replace=False)
df_with_missing.loc[missing_indices, '単価'] = np.nan

missing_indices2 = np.random.choice(df_with_missing.index, size=5, replace=False)
df_with_missing.loc[missing_indices2, 'カテゴリ'] = np.nan

# 1. 欠損値の確認
print("1. 欠損値の確認:")
print(f"欠損値の数:\n{df_with_missing.isnull().sum()}")
print(f"\n欠損値の割合:\n{(df_with_missing.isnull().sum() / len(df_with_missing) * 100).round(2)}%")

# 2. 欠損値のある行を表示
print("\n2. 欠損値のある行:")
missing_rows = df_with_missing[df_with_missing.isnull().any(axis=1)]
print(f"欠損値のある行数: {len(missing_rows)}")
print(missing_rows[['商品名', 'カテゴリ', '単価', '売上金額']].head())

# 3. 欠損値の処理方法
print("\n3. 欠損値の処理:")

# 3.1 欠損値を含む行を削除
df_dropped = df_with_missing.dropna()
print(f"欠損値削除後の行数: {len(df_dropped)} (元: {len(df_with_missing)})")

# 3.2 欠損値を平均値で埋める
df_filled_mean = df_with_missing.copy()
df_filled_mean['単価'] = df_filled_mean['単価'].fillna(df_filled_mean['単価'].mean())
print(f"単価の平均値: {df_filled_mean['単価'].mean():.2f}")

# 3.3 欠損値を最頻値で埋める
df_filled_mode = df_with_missing.copy()
mode_category = df_filled_mode['カテゴリ'].mode()[0]  # 最頻値
df_filled_mode['カテゴリ'] = df_filled_mode['カテゴリ'].fillna(mode_category)
print(f"カテゴリの最頻値: {mode_category}")

print("\n欠損値処理後の確認:")
print(f"平均値埋め後の欠損値: {df_filled_mean.isnull().sum().sum()}")
print(f"最頻値埋め後の欠損値: {df_filled_mode.isnull().sum().sum()}")

## 8. 簡単なグラフ描画

pandasの`plot()`メソッドを使って、データを可視化してみましょう。

In [ ]:
# グラフ描画の準備
plt.figure(figsize=(15, 12))

# 1. カテゴリ別売上の棒グラフ
plt.subplot(2, 3, 1)
category_sales_sum = df_loaded.groupby('カテゴリ')['売上金額'].sum()
category_sales_sum.plot(kind='bar', color='skyblue')
plt.title('カテゴリ別売上合計')
plt.xlabel('カテゴリ')
plt.ylabel('売上金額')
plt.xticks(rotation=45)

# 2. 商品別売上の棒グラフ
plt.subplot(2, 3, 2)
product_sales_sum = df_loaded.groupby('商品名')['売上金額'].sum()
product_sales_sum.plot(kind='bar', color='lightgreen')
plt.title('商品別売上合計')
plt.xlabel('商品名')
plt.ylabel('売上金額')
plt.xticks(rotation=45)

# 3. 地域別売上の円グラフ
plt.subplot(2, 3, 3)
region_sales = df_loaded.groupby('地域')['売上金額'].sum()
region_sales.plot(kind='pie', autopct='%1.1f%%', startangle=90)
plt.title('地域別売上構成比')
plt.ylabel('')

# 4. 売上金額のヒストグラム
plt.subplot(2, 3, 4)
df_loaded['売上金額'].hist(bins=20, alpha=0.7, color='orange')
plt.title('売上金額の分布')
plt.xlabel('売上金額')
plt.ylabel('頻度')

# 5. 単価と数量の散布図
plt.subplot(2, 3, 5)
plt.scatter(df_loaded['単価'], df_loaded['数量'], alpha=0.6, color='red')
plt.title('単価と数量の関係')
plt.xlabel('単価')
plt.ylabel('数量')

# 6. 日付別売上の折れ線グラフ
plt.subplot(2, 3, 6)
daily_sales = df_loaded.groupby('日付')['売上金額'].sum().sort_index()
daily_sales.plot(kind='line', marker='o', linewidth=2, markersize=4)
plt.title('日別売上推移')
plt.xlabel('日付')
plt.ylabel('売上金額')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

print("グラフを表示しました！")

# 個別のグラフも作成してみる
print("\n=== 追加のグラフ分析 ===")

# カテゴリ別の詳細分析
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 左：カテゴリ別平均売上
category_avg = df_loaded.groupby('カテゴリ')['売上金額'].mean()
category_avg.plot(kind='bar', ax=axes[0], color='purple', alpha=0.7)
axes[0].set_title('カテゴリ別平均売上')
axes[0].set_xlabel('カテゴリ')
axes[0].set_ylabel('平均売上金額')
axes[0].tick_params(axis='x', rotation=45)

# 右：カテゴリ別取引件数
category_count = df_loaded.groupby('カテゴリ').size()
category_count.plot(kind='bar', ax=axes[1], color='brown', alpha=0.7)
axes[1].set_title('カテゴリ別取引件数')
axes[1].set_xlabel('カテゴリ')
axes[1].set_ylabel('取引件数')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# 基本統計の確認
print("\n=== 最終的な分析サマリー ===")
print(f"総取引件数: {len(df_loaded):,}件")
print(f"総売上金額: {df_loaded['売上金額'].sum():,.0f}円")
print(f"平均売上金額: {df_loaded['売上金額'].mean():.0f}円")
print(f"最高売上: {df_loaded['売上金額'].max():,.0f}円")
print(f"最低売上: {df_loaded['売上金額'].min():,.0f}円")

print("\nカテゴリ別売上ランキング:")
category_ranking = df_loaded.groupby('カテゴリ')['売上金額'].sum().sort_values(ascending=False)
for i, (category, sales) in enumerate(category_ranking.items(), 1):
    print(f"{i}位: {category} - {sales:,.0f}円")

## 学習完了！🎉

お疲れ様でした！このノートブックでpandasの基本的なデータ操作・分析手法を学習しました。

### 習得したスキル
1. ✅ **pandasの基本設定とライブラリインポート**
2. ✅ **DataFrameの作成とデータ構造の理解**
3. ✅ **CSVファイルからのデータ読み込み**
4. ✅ **データの基本情報確認**（shape, head, info, describe）
5. ✅ **列・行の抽出とフィルタリング**
6. ✅ **groupbyを使ったデータ集計**
7. ✅ **データの並び替えと欠損値処理**
8. ✅ **pandasのplotメソッドによる可視化**

### 次のステップ
これらの基本スキルを身につけたので、次は以下のような発展的な内容に進むことができます：

1. **より高度なデータ分析**
   - 時系列データの分析
   - 顧客セグメンテーション
   - 売上予測

2. **外部データソースとの連携**
   - APIからのデータ取得
   - データベースとの連携
   - Webスクレイピングとの組み合わせ

3. **高度な可視化**
   - Seabornを使った統計的可視化
   - インタラクティブな可視化（Plotly）
   - ダッシュボードの作成

### 実践的な演習課題
1. 自分の興味のあるデータセット（Kaggleなど）をダウンロードして分析してみる
2. 複数のCSVファイルを結合して統合分析を行う
3. 時系列データで移動平均やトレンド分析を実装する

**素晴らしい学習でした！引き続きpandasを使ったデータ分析を楽しんでください！** 📊✨